In [ ]:
import base64
import branca.colormap as cm
import folium
from folium import IFrame
from glob import glob
from io import BytesIO
import json
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import matplotlib.image as mpimg
import os
import pandas as pd
import pickle
from PIL import Image, ImageOps
from scipy import ndimage
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import trange, tqdm

# Image details

In [ ]:
image_details = glob("/mnt/common/mncosta/datasets/mapillary/subjective_application_berlin/*.json")

details = []
for details_file in tqdm(image_details):
    aux1 = json.load(open(details_file))
    aux2 = {'image_i': aux1['image_number'],
            'Longitude': aux1['geometry']['coordinates'][0],
            'Latitude': aux1['geometry']['coordinates'][1],
            'captured_at': aux1['properties']['captured_at'],
            'compass_angle': aux1['properties']['compass_angle'],
            'id': aux1['properties']['id'],
            'is_pano': aux1['properties']['is_pano'],
            'sequence_id': aux1['properties']['sequence_id'],
            }

    details.append(aux2)

details = pd.DataFrame(details).set_index('image_i', drop=False)
details.index.name = None
details = details[details.is_pano == False] # Remove 360 images

In [ ]:
details_df = details[['Longitude', 'Latitude', 'image_i']]

# Image ranks

In [ ]:
ranks = pd.read_pickle('../outputs/saved/application_vgg_syn+ber.pt_results.pkl')

In [ ]:
ranks['image'] = ranks.image_left.str.replace('/mnt/common/mncosta/datasets/mapillary/subjective_application_berlin/', '').str.replace('_0.jpg', '').astype(int)
ranks = ranks.set_index('image', drop=False)
ranks.index.name = None
ranks_df = ranks[['rank_left', 'image', 'image_left']].rename(columns={'rank_left': 'rank', 'image_left': 'image_path'})

# Final Data

In [ ]:
df = pd.merge(details_df, ranks_df, left_index=True, right_index=True)

In [ ]:
scaler = MinMaxScaler()
df['rank_scaled'] = scaler.fit_transform(df['rank'].values.reshape(-1, 1)).reshape(1,-1)[0]

## City-wide Plot

In [ ]:
cmap = plt.cm.jet

m = folium.Map(width=1100, height=700,
               location=[df['Latitude'].mean(), df['Longitude'].mean()], 
               zoom_start=11, 
               tiles='CartoDB positron',
               zoom_control=False,
               scrollWheelZoom=False,
               dragging=False
              )

df.apply(lambda row: folium.Circle(
        location    =[row["Latitude"], row["Longitude"]],
        radius      =5, 
        fill_color  =rgb2hex(cmap(1-row['rank_scaled'])), 
        color       =rgb2hex(cmap(1-row['rank_scaled'])), 
        fill_opacity=.3,
        opacity     =.3,
    ).add_to(m), axis=1);

m

## Legend

In [ ]:
fig, axarr = plt.subplots(1,1, figsize=(15, 15));
rotated_img = ndimage.rotate(mpimg.imread('colormap.png'), -90);
axarr.imshow(rotated_img);
axarr.text(20, 10, '1 (Perceived Safe)', style='italic', color='k', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.0, 'pad': 1});
axarr.text(20, 500, '0 (Perceived Unsafe)', style='italic', color='k', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.0, 'pad': 1});
axarr.set_xticks([]);
axarr.set_yticks([]);
fig.savefig('application_colormap.png', dpi=fig.dpi)

## Show some SVI

In [ ]:
cmap = plt.cm.jet

# Base map
m = folium.Map(width=1100, height=700,
               location=[df['Latitude'].mean(), df['Longitude'].mean()], 
               zoom_start=13, 
               tiles='CartoDB positron',
#               zoom_control=False,
#               scrollWheelZoom=False,
#               dragging=False
              )

# Draw scores
df.apply(lambda row: folium.Circle(
        location    =[row["Latitude"], row["Longitude"]],
        radius      =5, 
        fill_color  =rgb2hex(cmap(1-row['rank_scaled'])), 
        color       =rgb2hex(cmap(1-row['rank_scaled'])), 
        fill_opacity=1.,
        opacity     =1.,
        tooltip='<b>{}</b'.format(row['rank_scaled']), 
    ).add_to(m), axis=1);

# Draw some SVI for a zone
df_ = df.copy()
df_ = df_[(df_.Latitude > 52.505) & (df_.Latitude < 52.525) & (df_.Longitude > 13.38) & (df_.Longitude < 13.42)].sample(50)
print(df_.shape)

for lat, lon, rank, imagename in zip(df_['Latitude'], df_['Longitude'], df_['rank_scaled'], df_['image_path']):
    im2 = ImageOps.equalize(Image.open(imagename), mask=None) # histogram equalize
    buffered = BytesIO()
    im2.save(buffered, format="JPEG")
    encoded = base64.b64encode(buffered.getvalue())
    
    #encoded = base64.b64encode(open(imagename, 'rb').read())
    html='<div><img src="data:image/jpeg;base64,{}" width="150" height="100"><br/><span>Score: {:.2f}</span></div>'.format
    resolution, width, height = 75, 50, 25
    iframe = IFrame(html(encoded.decode('UTF-8'), rank), width=170, height=140)
    popup = folium.Popup(iframe, max_width=200, show=True)
    marker = folium.Circle(location=[lat, lon], 
                           radius=10,
                           popup=popup, 
                           #icon=icon,
                           fill_color  =rgb2hex(cmap(1-rank)),
                           color       =rgb2hex(cmap(1-rank)), 
                           fill_opacity=1.,
                           opacity     =1.,
                           tooltip='<b>{0:.2f}</b'.format(rank), 
                          )
    marker.add_to(m)

m

In [ ]:
cmap = plt.cm.jet

# Base map
m = folium.Map(width=1100, height=700,
               location=[df['Latitude'].mean(), df['Longitude'].mean()], 
               zoom_start=13, 
               tiles='CartoDB positron',
#               zoom_control=False,
#               scrollWheelZoom=False,
#               dragging=False
              )

# Draw scores
df.apply(lambda row: folium.Circle(
        location    =[row["Latitude"], row["Longitude"]],
        radius      =15, 
        fill_color  =rgb2hex(cmap(1-row['rank_scaled'])), 
        color       =rgb2hex(cmap(1-row['rank_scaled'])), 
        fill_opacity=1.,
        opacity     =1.,
        tooltip='<b>{} / {}</b'.format(row['rank_scaled'], row['image_path'].replace('/mnt/common/mncosta/datasets/mapillary/subjective_application_berlin/', '')), 
    ).add_to(m), axis=1);

m

In [ ]:
#im = ImageOps.equalize(Image.open('/mnt/common/mncosta/datasets/mapillary/subjective_application_berlin/65151_0.jpg'), mask=None) # histogram equalize
im = Image.open('/mnt/common/mncosta/datasets/mapillary/subjective_application_berlin/65150_0.jpg')
im = ImageOps.scale(im, 0.25)
im